In [179]:
import pandas as pd

# Assuming you uploaded a file named "data.csv"
df = pd.read_csv('/Daily Household Transactions.csv')
print(df.head())


                  Date                   Mode        Category  \
0  20/09/2018 12:04:08                   Cash  Transportation   
1  20/09/2018 12:03:15                   Cash            Food   
2           19/09/2018  Saving Bank account 1    subscription   
3  17/09/2018 23:41:17  Saving Bank account 1    subscription   
4  16/09/2018 17:15:08                   Cash       Festivals   

               Subcategory                         Note  Amount  \
0                    Train         2 Place 5 to Place 0    30.0   
1                   snacks  Idli medu Vada mix 2 plates    60.0   
2                  Netflix         1 month subscription   199.0   
3  Mobile Service Provider            Data booster pack    19.0   
4             Ganesh Pujan                  Ganesh idol   251.0   

  Income/Expense Currency  
0        Expense      INR  
1        Expense      INR  
2        Expense      INR  
3        Expense      INR  
4        Expense      INR  


In [180]:
# Fill missing values
df.fillna("Unknown", inplace=True)

# Fill missing numerical values with median
for col in df.select_dtypes(include=["number"]).columns:
    df[col].fillna(df[col].median(), inplace=True)

# Drop any remaining missing values
df.dropna(inplace=True)

print("Missing values handled successfully!")


Missing values handled successfully!


In [181]:
# Convert text to lowercase
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

# Remove unnecessary spaces and special characters
df.columns = df.columns.str.strip().str.replace("[^a-zA-Z0-9_ ]", "", regex=True)

print("Data cleaned successfully!")


Data cleaned successfully!


In [182]:
df.columns


Index(['Date', 'Mode', 'Category', 'Subcategory', 'Note', 'Amount',
       'IncomeExpense', 'Currency'],
      dtype='object')

In [183]:
# Convert the 'Category' column to lowercase for easy matching
df['Category'] = df['Category'].fillna('').astype(str).str.lower()

# Define keywords for categorization
food_keywords = [
    'food', 'grocery', 'groceries', 'fruits', 'vegetables', 'restaurant',
    'takeout', 'snack', 'breakfast', 'lunch', 'dinner', 'beverages'
]

fixed_expense_keywords = [
    'rent', 'subscription', 'netflix', 'amazon', 'fees', 'recharge',
    'travel', 'bus', 'train', 'metro', 'gas', 'electricity', 'wifi',
    'household', 'utility', 'internet', 'hostel','transport'
]

# Categorization function using only the Category column
def categorize_from_category(category):
    if any(word in category for word in food_keywords):
        return 'Food'
    elif any(word in category for word in fixed_expense_keywords):
        return 'Fixed Expenses'
    else:
        return 'Miscellaneous'

# Apply the function
df['Category'] = df['Category'].apply(categorize_from_category)

# See the result
df['Category'].value_counts()

,count
Category,
Miscellaneous,922
Food,907
Fixed Expenses,632


In [184]:
df.head()

,Date,Mode,Category,Subcategory,Note,Amount,IncomeExpense,Currency
0,20/09/2018 12:04:08,cash,Fixed Expenses,train,2 place 5 to place 0,30.0,expense,inr
1,20/09/2018 12:03:15,cash,Food,snacks,idli medu vada mix 2 plates,60.0,expense,inr
2,19/09/2018,saving bank account 1,Fixed Expenses,netflix,1 month subscription,199.0,expense,inr
3,17/09/2018 23:41:17,saving bank account 1,Fixed Expenses,mobile service provider,data booster pack,19.0,expense,inr
4,16/09/2018 17:15:08,cash,Miscellaneous,ganesh pujan,ganesh idol,251.0,expense,inr


In [185]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Load the DataFrame (assuming already loaded as df)
df = df[['Note', 'Category','Date','Amount','IncomeExpense']].dropna()

# Encode categories
label_encoder = LabelEncoder()
df['CategoryEncoded'] = label_encoder.fit_transform(df['Category'])

# Tokenize text
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['Note'])

sequences = tokenizer.texts_to_sequences(df['Note'])
padded_sequences = pad_sequences(sequences, maxlen=20, padding='post')

# Prepare labels
labels = to_categorical(df['CategoryEncoded'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)


In [186]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=20),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')  # 3 categories
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_8 (Conv1D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_8               │ ?                           │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [187]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_test, y_test),
    batch_size=32
)


Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.4789 - loss: 1.0552 - val_accuracy: 0.7099 - val_loss: 0.7777
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7840 - loss: 0.6585 - val_accuracy: 0.7911 - val_loss: 0.5275
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8812 - loss: 0.4017 - val_accuracy: 0.8316 - val_loss: 0.4383
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9115 - loss: 0.3061 - val_accuracy: 0.8418 - val_loss: 0.4119
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9201 - loss: 0.2437 - val_accuracy: 0.8377 - val_loss: 0.4013
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9186 - loss: 0.2389 - val_accuracy: 0.8458 - val_loss: 0.4028
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9227 - loss: 0.2228 - val_accuracy: 0.8560 - val_loss: 0.4152
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9288 - loss: 0.2053 - val_accuracy: 0.8499 - v

In [188]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8477 - loss: 0.4467
Test Accuracy: 0.84


In [189]:

def predict_category(note):
    seq = tokenizer.texts_to_sequences([note])
    padded = pad_sequences(seq, maxlen=20, padding='post')
    pred = model.predict(padded)
    return label_encoder.inverse_transform([np.argmax(pred)])

# Example:
predict_category(" Food ")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step


array(['Food'], dtype=object)

In [190]:

predict_category("Ordered pizza ")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


array(['Food'], dtype=object)

In [191]:

predict_category("Purchased books for reading")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step


array(['Miscellaneous'], dtype=object)

In [192]:

predict_category("Paid for food delivery from Swiggy")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


array(['Food'], dtype=object)

In [193]:
predict_category("Biryani in lunch")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


array(['Food'], dtype=object)

In [194]:
predict_category("Vegetables")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


array(['Food'], dtype=object)

In [195]:





predict_category("Netflix subscription renewed")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step


array(['Fixed Expenses'], dtype=object)

In [196]:
predict_category("Paid monthly rent")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step


array(['Miscellaneous'], dtype=object)

In [197]:
predict_category("Yearly tuition fees submitted")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step


array(['Miscellaneous'], dtype=object)

In [198]:
predict_category("Bought grocery")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


array(['Food'], dtype=object)

In [199]:

predict_category("Ordered pizza for party")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


array(['Food'], dtype=object)

In [200]:

predict_category("Dinner at Barbeque Nation")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


array(['Food'], dtype=object)

In [201]:

predict_category("Vegetables and fruits")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


array(['Food'], dtype=object)

In [202]:
df

,Note,Category,Date,Amount,IncomeExpense,CategoryEncoded
0,2 place 5 to place 0,Fixed Expenses,20/09/2018 12:04:08,30.0,expense,0
1,idli medu vada mix 2 plates,Food,20/09/2018 12:03:15,60.0,expense,1
2,1 month subscription,Fixed Expenses,19/09/2018,199.0,expense,0
3,data booster pack,Fixed Expenses,17/09/2018 23:41:17,19.0,expense,0
4,ganesh idol,Miscellaneous,16/09/2018 17:15:08,251.0,expense,2
...,...,...,...,...,...,...
2456,share jeep - place t base to top,Fixed Expenses,1/1/2015,20.0,expense,0
2457,share auto - place h to place t base,Fixed Expenses,1/1/2015,20.0,expense,0
2458,bus - brc to place h,Fixed Expenses,1/1/2015,30.0,expense,0
2459,tea,Food,1/1/2015,10.0,expense,1


In [203]:
df['Category'].value_counts()


,count
Category,
Miscellaneous,922
Food,907
Fixed Expenses,632


In [204]:
print(predict_category("Netflix subscription renewed"))
print(predict_category("Paid rent"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
['Fixed Expenses']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
['Miscellaneous']


In [205]:
df.count()

,0
Note,2461
Category,2461
Date,2461
Amount,2461
IncomeExpense,2461
CategoryEncoded,2461


In [206]:
df


,Note,Category,Date,Amount,IncomeExpense,CategoryEncoded
0,2 place 5 to place 0,Fixed Expenses,20/09/2018 12:04:08,30.0,expense,0
1,idli medu vada mix 2 plates,Food,20/09/2018 12:03:15,60.0,expense,1
2,1 month subscription,Fixed Expenses,19/09/2018,199.0,expense,0
3,data booster pack,Fixed Expenses,17/09/2018 23:41:17,19.0,expense,0
4,ganesh idol,Miscellaneous,16/09/2018 17:15:08,251.0,expense,2
...,...,...,...,...,...,...
2456,share jeep - place t base to top,Fixed Expenses,1/1/2015,20.0,expense,0
2457,share auto - place h to place t base,Fixed Expenses,1/1/2015,20.0,expense,0
2458,bus - brc to place h,Fixed Expenses,1/1/2015,30.0,expense,0
2459,tea,Food,1/1/2015,10.0,expense,1


In [207]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

def train_and_forecast_next_month(df, input_month):
    # Ensure datetime and amount columns are correct
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
    df.dropna(subset=['Date', 'Amount'], inplace=True)

    # Monthly aggregation
    monthly_expense = df.groupby(df['Date'].dt.to_period('M'))['Amount'].sum().to_timestamp()

    # Convert input_month to datetime
    try:
        input_date = pd.to_datetime(input_month)
    except:
        return {"❌ Error": "Please enter date in YYYY-MM format."}

    if input_date not in monthly_expense.index:
        return {"❌ Error": f"No data available for {input_month}"}

    # Train data up to input_month only
    train_data = monthly_expense[monthly_expense.index <= input_date]

    if len(train_data) < 3:
        return {"❌ Error": "Not enough data to train model."}

    # Train ARIMA model
    model = ARIMA(train_data, order=(1, 1, 1))
    model_fit = model.fit()

    # Forecast next month
    prediction = model_fit.forecast(steps=1)[0]
    last_expense = train_data[-1]

    next_month = (input_date + pd.DateOffset(months=1)).strftime('%B %Y')
    lower_bound = max(0, prediction - 5000)
    upper_bound = prediction + 5000

    return {
        "📅 Current Month": input_date.strftime('%B %Y'),
        "💸 Actual Expense": f"₹{last_expense:.2f}",
        f"📈 Forecast for {next_month}": f"₹{prediction:.2f}",
        "📊 Expected Range": f"₹{lower_bound:.2f} – ₹{upper_bound:.2f}"
    }


In [208]:
result = train_and_forecast_next_month(df, '2017-06')
for k, v in result.items():
    print(f"{k}: {v}")


📅 Current Month: June 2017
💸 Actual Expense: ₹271335.50
📈 Forecast for July 2017: ₹168026.95
📊 Expected Range: ₹163026.95 – ₹173026.95


In [209]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.dropna(subset=['Date'], inplace=True)
df['YearMonth'] = df['Date'].dt.to_period('M').astype(str)
# Convert income/expense to lowercase
df['IncomeExpense'] = df['IncomeExpense'].fillna('').str.lower()

# Assign signed values
df['SignedAmount'] = df.apply(
    lambda row: row['Amount'] if 'income' in row['IncomeExpense'] else -row['Amount'], axis=1
)

# Calculate monthly net savings
monthly = df.groupby('YearMonth')['SignedAmount'].sum().reset_index()
monthly.rename(columns={'SignedAmount': 'NetSavings'}, inplace=True)

In [210]:
import pandas as pd

# Load and preprocess data
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.dropna(subset=['Date'], inplace=True)
df['YearMonth'] = df['Date'].dt.to_period('M').astype(str)
df['IncomeExpense'] = df['IncomeExpense'].fillna('').str.lower()

# Assign signed values based on income/expense
df['SignedAmount'] = df.apply(
    lambda row: row['Amount'] if 'income' in row['IncomeExpense'] else -row['Amount'], axis=1
)

# 📌 Define Investment Label Logic
def label_investment_plan(savings):
    if savings < 2000:
        return 'Emergency Fund'
    elif savings < 5000:
        return 'Small SIP'
    elif savings < 10000:
        return 'SIP + Mutual Fund'
    elif savings < 20000:
        return 'SIP + Stocks + Gold'
    else:
        return 'Full Diversification'

# 🚀 Function to Get Investment Suggestion for a Given Month
def get_investment_suggestion(month: str):
    monthly_savings = df[df['YearMonth'] == month]['SignedAmount'].sum()
    suggestion = label_investment_plan(monthly_savings)
    return {
        "Month": month,
        "Net Savings": f"₹{monthly_savings:.2f}",
        "Investment Suggestion": suggestion
    }

# 🔍 Example Usage
result = get_investment_suggestion("2018-01")
print(result)


{'Month': '2018-01', 'Net Savings': '₹-77022.04', 'Investment Suggestion': 'Emergency Fund'}


In [211]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Step 1: Prepare monthly data
monthly = df.groupby('YearMonth')['SignedAmount'].sum().reset_index()
monthly.columns = ['YearMonth', 'NetSavings']

# Step 2: Label data with your existing rule
monthly['InvestmentPlan'] = monthly['NetSavings'].apply(label_investment_plan)

# Step 3: Encode labels for ML
le = LabelEncoder()
monthly['PlanEncoded'] = le.fit_transform(monthly['InvestmentPlan'])

# Step 4: Train model
X = monthly[['NetSavings']]
y = monthly['PlanEncoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Step 5: Evaluate
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))


                      precision    recall  f1-score   support

      Emergency Fund       1.00      1.00      1.00         1
Full Diversification       1.00      1.00      1.00         8

            accuracy                           1.00         9
           macro avg       1.00      1.00      1.00         9
        weighted avg       1.00      1.00      1.00         9



In [212]:
def predict_investment_plan_ml(savings_amount):
    pred_label = rf_model.predict([[savings_amount]])[0]
    return le.inverse_transform([pred_label])[0]

# Example: Predict for ₹8200
prediction = predict_investment_plan_ml(8200)
print(f"Predicted Plan: {prediction}")


Predicted Plan: Emergency Fund


In [213]:
from tensorflow.keras.models import load_model

# Save the model
model.save('cnn_model.h5')


In [214]:
import joblib

# Save tokenizer
joblib.dump(tokenizer, 'tokenizer.pkl')

# Save label encoder
joblib.dump(label_encoder, 'label_encoder.pkl')


['label_encoder.pkl']

In [215]:
joblib.dump(rf_model, 'rf_model.pkl')                # Forecasting model



['rf_model.pkl']

In [216]:
import pickle

# your fitted ARIMA model is called `fitted_model`


In [217]:
with open('arima_model.pkl', 'wb') as f:
    pickle.dump(result, f)


In [218]:
from google.colab import files

# CNN model
files.download('cnn_model.h5')           # or files.download('cnn_model.h5')
files.download('tokenizer.pkl')
files.download('label_encoder.pkl')

# Random Forest model
files.download('rf_model.pkl')


# ARIMA model
files.download('arima_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>